In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated

load_dotenv()


True

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_deepseek import ChatDeepSeek

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_groq import ChatGroq

from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import InMemorySaver

from langchain_core.messages import HumanMessage, BaseMessage


In [ ]:
LLMS = {}

LLMS["gemini"] = ChatGoogleGenerativeAI(
    api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash"
)

LLMS["deepseek"] = ChatDeepSeek(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    api_base=os.getenv("OPENROUTER_BASE_URL"),
    model="deepseek/deepseek-chat-v3-0324:free",
)

LLMS["moonshotai"] = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"), model="moonshotai/kimi-k2-instruct"
)


In [ ]:
llm: ChatDeepSeek = LLMS["deepseek"]


In [ ]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


In [ ]:
def chat_node(state: ChatState):
    messages = state["messages"]
    result = llm.invoke(messages)
    state["messages"] = [result.content]

    return state


In [ ]:
graph = StateGraph(ChatState)

graph.add_node("chat_node", chat_node)

graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

checkpointer = InMemorySaver()

chatbot = graph.compile(checkpointer=checkpointer)


In [ ]:
thread_id = 1
config = {"configurable": {"thread_id": thread_id}}
while True:
    user_msg = input(">>>Enter your message: ")

    print("User Message:", user_msg)

    if user_msg.strip().lower() in ["quit", "exit", "bye"]:
        break

    initial_state = {"messages": [HumanMessage(content=user_msg)]}

    final_state = chatbot.invoke(initial_state, config=config)

    print("AI Message:", final_state["messages"][-1].content)


User Message: Hey there, I'm sourav
AI Message: Hey Sourav! 👋 What's up? How can I help you today? 😊
User Message: what was my name?
AI Message: Your name is **Sourav**! 😊 You mentioned it right at the beginning with *"Hey there, I'm Sourav."*  

Is there something specific you'd like help with, Sourav? 🚀
User Message: What's the use case of python language
AI Message: Python is a versatile, high-level programming language with a wide range of use cases across industries. Here are some of its most popular applications:  

### **1. Web Development**  
   - **Frameworks:** Django, Flask, FastAPI  
   - Used to build scalable websites (e.g., Instagram, Pinterest).  

### **2. Data Science & Analytics**  
   - **Libraries:** Pandas, NumPy, SciPy  
   - Helps clean, analyze, and visualize data for insights.  

### **3. Machine Learning & AI**  
   - **Libraries:** TensorFlow, PyTorch, Scikit-learn  
   - Powers AI models (chatbots, autonomous systems, recommendation engines).  

### **4. Au